In [ ]:
# Dependencies
# conda create -n SEPA_env
# conda activate SEPA_env
# pip install kiwis-pie jupyterlab geopandas folium 
from datetime import date
from kiwis_pie import KIWIS
import geopandas as gp
import folium
import matplotlib.pyplot as plt

In [ ]:
# Make initial connection using base API address
k = KIWIS('https://timeseries.sepa.org.uk/KiWIS/KiWIS')

In [ ]:
file_location = 'C:\\Users\\crs2\\Desktop\\SEPA_API_tests\\SEPA_KISTER_API_AV\\forth_catchment.geojson'
rio = gp.read_file(file_location)

In [ ]:
rio.plot()

In [ ]:
# List of SEPA stations
station_list = k.get_station_list()

In [ ]:
station_list

In [ ]:
# get rid of stations without lat lon
station_list = station_list.loc[station_list['station_latitude'] != '',]

# Turn stations into geopandas data frame
station_list = gp.GeoDataFrame(
    station_list, geometry=gp.points_from_xy(station_list.station_longitude, station_list.station_latitude))
station_list = station_list.set_crs(epsg=4326)

In [ ]:
# Spatially join with region of interest using a point in polygon 
rio_stations = gp.sjoin(station_list,rio)
# Drop nuisance columns
rio_stations = rio_stations.drop(columns=['index_right','fid'])

In [ ]:
# Plot 10 examples of stations within the Forth
rio_stations.head(70)

In [ ]:
# Plot
# base = station_list.plot(marker='o', color='blue', markersize=1,figsize=(10,10))
# base = rio.plot(ax=base, color='#ffffff00', edgecolor='black')
# rio_stations.plot(ax=base, marker='*', color='red', markersize=6)

### Plot locations of stations

In [ ]:
# create base map
map = folium.Map(location=[56.015991, -3.481558], zoom_start=9, control_scale=True)
folium.GeoJson(file_location, name="geojson").add_to(map)
# iterate over stations and plot locations with markers
for index, location_info in rio_stations.iterrows():
    folium.Marker([location_info["station_latitude"], location_info["station_longitude"]], popup=location_info["station_name"]).add_to(map)
map

### List of available variables

In [ ]:
# station parameters 'Flow', 'Rain', 'Level'
parameters_list = k.get_parameter_list(station_id = rio_stations.station_id.values)
# List all relevant parameters
parameters_list['stationparameter_name'].unique().tolist()

### Filter by river level and plot

In [ ]:
# We are interested in just water level for this exercise so return only those results
parameters_list = k.get_parameter_list(station_id = rio_stations.station_id.values,stationparameter_name='Level')
print(parameters_list.shape)
parameters_list.head()

In [ ]:
# Select only station data for relevant ids within the forth
rio_stations_par = rio_stations.loc[rio_stations['station_id'].isin(parameters_list['station_id'])]
#rio_stations_par.to_excel("C:\\Users\\crs2\\Desktop\\SEPA_API_tests\\Level.xlsx")
# plot
map = folium.Map(location=[56.015991, -3.481558], zoom_start=9, control_scale=True)
# iterate over stations and plot locations with markers
for index, location_info in rio_stations_par.iterrows():
    folium.Marker([location_info["station_latitude"], location_info["station_longitude"]], popup=location_info["station_name"]).add_to(map)
map

### Filter stations that measure rainfall and plot

In [ ]:
# We are interested in just water level for this exercise so return only those results
parameters_list = k.get_parameter_list(station_id = rio_stations.station_id.values,stationparameter_name='Rain')
# Select only station data for relevant ids within the forth
rio_stations_par = rio_stations.loc[rio_stations['station_id'].isin(parameters_list['station_id'])]
# plot
map = folium.Map(location=[56.015991, -3.481558], zoom_start=9, control_scale=True)
# iterate over stations and plot locations with markers
for index, location_info in rio_stations_par.iterrows():
    folium.Marker([location_info["station_latitude"], location_info["station_longitude"]], popup=location_info["station_name"]).add_to(map)
map

### Tidal Level

In [ ]:
# We are interested in just water level for this exercise so return only those results
parameters_list = k.get_parameter_list(station_id = rio_stations.station_id.values,stationparameter_name='TidalLevel')
# Select only station data for relevant ids within the forth
rio_stations_par = rio_stations.loc[rio_stations['station_id'].isin(parameters_list['station_id'])]
# plot
map = folium.Map(location=[56.015991, -3.481558], zoom_start=9, control_scale=True)
# iterate over stations and plot locations with markers
for index, location_info in rio_stations_par.iterrows():
    folium.Marker([location_info["station_latitude"], location_info["station_longitude"]], popup=location_info["station_name"]).add_to(map)
map

### Ground water level

In [ ]:
# We are interested in just water level for this exercise so return only those results
parameters_list = k.get_parameter_list(station_id = rio_stations.station_id.values,stationparameter_name='GroundwaterLevel')
# Select only station data for relevant ids within the forth
rio_stations_par = rio_stations.loc[rio_stations['station_id'].isin(parameters_list['station_id'])]
# plot
map = folium.Map(location=[56.015991, -3.481558], zoom_start=9, control_scale=True)
# iterate over stations and plot locations with markers
for index, location_info in rio_stations_par.iterrows():
    folium.Marker([location_info["station_latitude"], location_info["station_longitude"]], popup=location_info["station_name"]).add_to(map)
map

### Explore which data we can get for each variable measured

In [ ]:
def get_available_time_series(stationparameter_name):
    # Check out the different time series interval options for a specific station parameter names
    parameters_list =k.get_parameter_list(station_id = rio_stations.station_id.values,stationparameter_name=stationparameter_name)
    # Select only station data for relevant ids within the forth
    rio_stations_par = rio_stations.loc[rio_stations['station_id'].isin(parameters_list['station_id'])]
    time_series_options = k.get_timeseries_list(station_id = rio_stations_par.station_id,stationparameter_name=stationparameter_name)['ts_name'].unique().tolist()
    return(time_series_options)

In [ ]:
stationparameter_name = 'Level'
get_available_time_series(stationparameter_name)

In [ ]:
stationparameter_name = 'Rain'
get_available_time_series(stationparameter_name)

In [ ]:
stationparameter_name = 'Flow'
get_available_time_series(stationparameter_name)

In [ ]:
stationparameter_name = 'TidalLevel'
get_available_time_series(stationparameter_name)

In [ ]:
stationparameter_name = 'GroundwaterLevel'
get_available_time_series(stationparameter_name)

In [ ]:
# # Look at the documentation for defining returned column headers https://kiwis-pie.readthedocs.io/en/latest/api/kiwis_pie.html#module-kiwis_pie.kiwis
# headers=['station_name','station_no','station_id','ts_id','ts_name','parametertype_id','parametertype_name',
#          'stationparameter_name','stationparameter_longname','ts_density','ts_unitname','ts_unitsymbol','coverage']

# # return ts ids for stations, parameters and time series interval options selected
# #ts_frame = k.get_timeseries_list(station_id = rio_stations_par.station_id, stationparameter_name='Level',ts_name='Day.Mean',return_fields=headers)
# ts_frame = k.get_timeseries_list(station_id = rio_stations_par.station_id, stationparameter_name='Rain',ts_name='Day.Mean',return_fields=headers)

# # Just to check there are 50 time series datasets available for the stations, parameters and time series interval options selected
# print('Stations available for this variable: '+str(ts_frame.shape[0]))

# # Using Callander as demonstrator return ts_id of interest
# ts_of_interest = ts_frame.loc[ts_frame['station_name'] == 'Callander']['ts_id'].values[0] # time series ID
# # Get time series data
# timeseries_dataset = k.get_timeseries_values(ts_id = ts_of_interest, to = date(2022,5,31), **{'from': date(2018,1,1)})

# # Plot data - you can automate the addition of labels from metadata  
# ax = timeseries_dataset.plot(title='River water level Callander',figsize=(16,10))
# ax.set(xlabel="Date", ylabel="River Water level (m)")

### Get time series data for a given variable of a given station

In [ ]:
rio_stations

In [ ]:
def get_SEPA_time_series(headers,parameter_name,ts_type,station,start,end):
    # Filter stations that measure parameter_name
    parameters_list =k.get_parameter_list(station_id = rio_stations.station_id.values,stationparameter_name=parameter_name)
    # Select only station data within the forth
    rio_stations_par = rio_stations.loc[rio_stations['station_id'].isin(parameters_list['station_id'])]
    # return timeseries 'ts_type' for parameter 'parameter_name' and time series interval options selected
    ts_frame = k.get_timeseries_list(station_id = rio_stations_par.station_id, stationparameter_name=parameter_name,ts_name=ts_type,return_fields=headers)

    # check No. of time series datasets available for the stations, parameters and time series interval options selected
    print('Stations available for '+parameter_name+': '+str(ts_frame.shape[0]))

    # filter dataframe by station name
    ts_of_interest = ts_frame.loc[ts_frame['station_name'] == station]['ts_id'].values[0] # time series ID
    # Get time series data
    timeseries_dataset = k.get_timeseries_values(ts_id = ts_of_interest, to = end, **{'from': start})
    return(timeseries_dataset)

In [ ]:
# Plot in a separate window
#%matplotlib qt

In [ ]:
# Look at the documentation for defining returned column headers https://kiwis-pie.readthedocs.io/en/latest/api/kiwis_pie.html#module-kiwis_pie.kiwis
headers=['station_name','station_no','station_id','ts_id','ts_name','parametertype_id','parametertype_name',
         'stationparameter_name','stationparameter_longname','ts_density','ts_unitname','ts_unitsymbol','coverage']

station = 'Gargunnock'# 
#station = 'Strathyre'
#station = 'Craigforth'
ts_type = 'Day.Mean'
start = date(2015,10,1)
end=date.today()#date(2022,9,13) 

parameter_name = 'Level'
level_time_series = get_SEPA_time_series(headers,parameter_name,ts_type,station,start,end)

fig,((ax,ax1))=plt.subplots(nrows=2,ncols=1,sharex=True,figsize=(18,10))
level_time_series.plot(ax=ax,style="-o")
ax.set_title(parameter_name+' - '+station)
ax.set(ylabel="River Water level (m)")
ax.grid(True)
#station = 'Westwood Farm Lane'
#station='Sterling Mills Gauging Station'
#station = 'Bridge of Teith'
#station = 'Glenochil'
#parameter_name ='Flow' #
#parameter_name ='Rain' #
#ts_type = 'Day.Total'
rain_time_series = get_SEPA_time_series(headers,parameter_name,ts_type,station,start,end)

rain_time_series.plot(ax=ax1,style="-o")
ax1.set_title(parameter_name+' - '+station)
#ax1.set(xlabel="Date", ylabel="Rainfall")
ax1.set(ylabel="River Water level (m)")
ax1.grid(True)
plt.tight_layout()

In [ ]:
import pandas as pd
pd.options.display.max_rows = 2000 #Changes the number of rows diplayed (default is 60)
rain_time_series.head(2000)

In [ ]:
fig,ax1=plt.subplots()
ax1.scatter(x=level_time_series,y=rain_time_series,c='DarkBlue')
ax1.set(xlabel="River level (m)", ylabel="River flow")
plt.title('River level vs river flow')

In [ ]:
g = rain_time_series.groupby(pd.Grouper(freq="M")).median()
g.plot.bar(figsize=(18,9))

### Driest month

In [ ]:
g[g.Value==g.min()[0]]

In [ ]:
rain_time_series.median()

In [ ]:
#rain_time_series['Value']
rain_time_series[rain_time_series.Value < 0.8]

In [ ]:
station_name = 'Bridge of Allan'
parameters_available_by_station_name = k.get_parameter_list(station_name = station_name)
#parameters_available_by_station_name['stationparameter_name'].unique().tolist()
parameters_available_by_station_name

In [ ]:
station_name='Gargunnock'
parameters_available_by_station_name = k.get_parameter_list(station_name = station_name)
#parameters_available_by_station_name['stationparameter_name'].unique().tolist()
parameters_available_by_station_name

In [ ]:
station_name='Callander'
parameters_available_by_station_name = k.get_parameter_list(station_name = station_name)
#parameters_available_by_station_name['stationparameter_name'].unique().tolist()
parameters_available_by_station_name

In [ ]:
station_name='Strathyre'
parameters_available_by_station_name = k.get_parameter_list(station_name = station_name)
#parameters_available_by_station_name['stationparameter_name'].unique().tolist()
parameters_available_by_station_name

In [ ]:
station_name='Musselburgh Tidal'
parameters_available_by_station_name = k.get_parameter_list(station_name = station_name)
#parameters_available_by_station_name['stationparameter_name'].unique().tolist()
parameters_available_by_station_name

In [ ]:
station_name='Glenochil'
parameters_available_by_station_name = k.get_parameter_list(station_name = station_name)
#parameters_available_by_station_name['stationparameter_name'].unique().tolist()
parameters_available_by_station_name

In [ ]:
station_name='Westwood Farm Lane'
parameters_available_by_station_name = k.get_parameter_list(station_name = station_name)
#parameters_available_by_station_name['stationparameter_name'].unique().tolist()
parameters_available_by_station_name

In [ ]:
station_name='Sterling Mills Gauging Station'
parameters_available_by_station_name = k.get_parameter_list(station_name = station_name)
#parameters_available_by_station_name['stationparameter_name'].unique().tolist()
parameters_available_by_station_name

In [ ]:
station_name='Bridge of Teith'
parameters_available_by_station_name = k.get_parameter_list(station_name = station_name)
#parameters_available_by_station_name['stationparameter_name'].unique().tolist()
parameters_available_by_station_name


In [ ]:
station_name='craigforth'
parameters_available_by_station_name = k.get_parameter_list(station_name = station_name)
#parameters_available_by_station_name['stationparameter_name'].unique().tolist()
parameters_available_by_station_name